# 3D visualization of ORCS maps

In [ ]:
import pyvista as pv

In [ ]:
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

In [ ]:
def load_map(line, param, comp):
    if comp == 1: comp =''
    else: comp = f'_{comp}'
    return pyfits.getdata(f'IC1613_Field1_SN3/MAPS/IC1613_Field1_SN3.LineMaps.map.{line}{comp}.1x1.{param}.fits').astype(float)

def load_maps(line, param, ncomp):

    dfs = list()
    for icomp in np.arange(ncomp) + 1:
        idf = pd.DataFrame(columns=['x', 'y', param])
        imap = load_map(line, param, icomp)
        ix, iy = np.nonzero(~np.isnan(imap))
        idat = imap[~np.isnan(imap)]
        idf['x'] = ix
        idf['y'] = iy
        idf[param] = idat
        dfs.append(idf)
    df = pd.concat(dfs, ignore_index=True)
    return df

vels = load_maps(6563, 'velocity', 3)
fluxes = load_maps(6563, 'flux', 3)

In [ ]:
ok = fluxes.flux > np.nanpercentile(fluxes.flux, 0)
ok &= (vels.velocity < 0) & (vels.velocity > -300)
vels = vels[ok]
fluxes = fluxes[ok]
vel_corr = vels.velocity #np.nanmedian(vels.velocity)
positions = np.squeeze(np.dstack((vel_corr, vels.x - np.nanmedian(vels.x), vels.y - np.nanmedian(vels.y))))
velocity_cloud = pv.PolyData(positions)
velocity_cloud[r'Flux'] = fluxes.flux
clim = np.nanpercentile(fluxes.flux, [2, 98])
opacity = fluxes.flux.values.copy()
opacity -= opacity.min()
opacity /= opacity.max()
plotter = pv.Plotter(window_size=(1200, 1200))
plotter.set_background('black')
plotter.add_mesh(velocity_cloud, 
           lighting=False, 
           show_edges=True,
           show_scalar_bar=False, 
           render_points_as_spheres=True, 
           ambient=0.5, point_size=5, 
           interpolate_before_map=False, 
           opacity=opacity**0.5, 
           cmap='plasma', clim=clim,)
plotter.show()
plotter.close()